<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Filtering" data-toc-modified-id="Filtering-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Filtering</a></span></li><li><span><a href="#Count-table" data-toc-modified-id="Count-table-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Count table</a></span></li><li><span><a href="#Trees" data-toc-modified-id="Trees-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Trees</a></span></li><li><span><a href="#Checks" data-toc-modified-id="Checks-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Checks</a></span></li></ul></li><li><span><a href="#Preprocessing-counts" data-toc-modified-id="Preprocessing-counts-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Preprocessing counts</a></span></li><li><span><a href="#Prune-tree-&amp;-make-matrix" data-toc-modified-id="Prune-tree-&amp;-make-matrix-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Prune tree &amp; make matrix</a></span><ul class="toc-item"><li><span><a href="#Phylo" data-toc-modified-id="Phylo-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Phylo</a></span></li><li><span><a href="#Trait" data-toc-modified-id="Trait-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Trait</a></span></li></ul></li><li><span><a href="#Features" data-toc-modified-id="Features-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Features</a></span><ul class="toc-item"><li><span><a href="#Train-test-split" data-toc-modified-id="Train-test-split-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Train-test split</a></span></li><li><span><a href="#Writing" data-toc-modified-id="Writing-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Writing</a></span></li></ul></li><li><span><a href="#MDeep" data-toc-modified-id="MDeep-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>MDeep</a></span><ul class="toc-item"><li><span><a href="#Convert-to-numpy" data-toc-modified-id="Convert-to-numpy-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Convert to numpy</a></span></li><li><span><a href="#Train:-age" data-toc-modified-id="Train:-age-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Train: age</a></span></li><li><span><a href="#Train:-westernized" data-toc-modified-id="Train:-westernized-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Train: westernized</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Sandbox" data-toc-modified-id="Sandbox-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Sandbox</a></span></li></ul></div>

# Goal

* Try [MDeep](https://github.com/lichen-lab/MDeep) on global metagenome dataset

# Var

In [336]:
# working dir
work_dir = '/ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/ML/MDeep/'

# bracken counts
brk_file = '/ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/diversity/bracken_filt.qs'

# GTDBr89 genome metadata
genome_meta_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/metadata_1per-GTDB-Spec_gte50comp-lt5cont_wtaxID_wPath.tsv'

# sample metadata
base_in_dir = '/ebio/abt3_projects/small_projects/nyoungblut/public_data_retireval/'
metadata_file = file.path(base_in_dir, 'CurMetDat-metagenomes', 'files', 'metadata', 'Filtered_CurMetDat_f.tsv')

# trees
## GTDB phylogeny
phy_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/phylogeny/ar122-bac120_r89_1per-GTDB-Spec_gte50comp-lt5cont_rn.nwk'
## trait phylogeny
phy_trt_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/genomes/LLG//phenotype/predictions_flat_majority-votes_combined_jaccard_rn.nwk'

# params
threads = 8
my_seed = 68372

# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(ape)
library(LeyLabRMisc)

In [11]:
make_dir(work_dir)
df.dims()

Created directory: /ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/ML/MDeep/ 


# Load

## Metadata

In [173]:
# sample metadata
meta = Fread(metadata_file) %>%
    mutate.(Sample = gsub('-', '.', Sample)) %>%
    filter.(number_reads >= 5e5)
meta

dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
AsnicarF_2017,MV_FEM1_t1Q14,MV_FEM1,stool,NA,control,healthy,NA,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
AsnicarF_2017,MV_FEM2_t1Q14,MV_FEM2,stool,NA,control,healthy,NA,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
ZellerG_2014,CCIS82944710ST.20.0,FR-730,stool,NA,control,healthy,38,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
ZellerG_2014,CCIS93040568ST.20.0,FR-682,stool,NA,control,healthy,65,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


## Filtering

In [174]:
meta = meta %>%
    filter.(!is.na(age))
meta

dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
Bengtsson-PalmeJ_2015,TRAVELRES1,TRAVELRES1_2,stool,no,control,healthy,25,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
Bengtsson-PalmeJ_2015,TRAVELRES10,TRAVELRES9_10,stool,no,control,healthy,34,adult,female,⋯,NA,NA,LKA,NA,NA,NA,NA,1,1,TRUE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
ZellerG_2014,CCIS82944710ST.20.0,FR-730,stool,NA,control,healthy,38,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
ZellerG_2014,CCIS93040568ST.20.0,FR-682,stool,NA,control,healthy,65,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


## Count table

In [175]:
# Bracken count table
brk = qs::qread(brk_file, nthreads=threads) %>%
    mutate.(name = gsub(' ', '_', name))
brk 

dataset,name,taxonomy_id,taxonomy_lvl,Domain,Phylum,Class,Order,Family,Genus,Species,Sample,Abundance
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
CosteaPI_2017,s__Caecibacter_massiliensis,108259,S,Bacteria,Firmicutes_C,Negativicutes,Veillonellales,Megasphaeraceae,Caecibacter,Caecibacter massiliensis,halbarad.11.0.0,0
CosteaPI_2017,s__UBA1777_sp002350825,178550,S,Bacteria,Firmicutes_A,Clostridia,Oscillospirales,Oscillospiraceae,UBA1777,UBA1777 sp002350825,halbarad.11.0.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YuJ_2015,s__Serratia_grimesii,121568,S,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Serratia,Serratia grimesii,SZAXPI017592.142,0
YuJ_2015,s__Bacteroides_A_sp002161565,119189,S,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides_A,Bacteroides_A sp002161565,SZAXPI017592.142,0


In [176]:
# Filtering to match 
x = intersect(meta %>% pull.(Sample),
              brk %>% pull.(Sample))
meta = meta %>%
    filter.(Sample %in% x)
meta %>% unique_n('Samples', Sample)

brk = brk %>%
    filter.(Sample %in% x)
brk %>% unique_n('Samples', Sample)

No. of unique Samples: 1929 
No. of unique Samples: 1929 


## Trees

In [177]:
phy = read.tree(phy_file)
phy


Phylogenetic tree with 23360 tips and 23359 internal nodes.

Tip labels:
	s__Halorubrum_sp000296615, s__Halorubrum_distributum, s__Halorubrum_trapanicum, s__Halorubrum_tropicale, s__Halorubrum_coriense, s__Halorubrum_ezzemoulense, ...
Node labels:
	100.0, d__Archaea100.0, 97.0, 99.0, 100.0, 100.0-p__Halobacterota, ...

Rooted; includes branch lengths.

In [178]:
trt = read.tree(phy_trt_file)
trt


Phylogenetic tree with 23360 tips and 23359 internal nodes.

Tip labels:
	s__Mikella_endobia, s__Prochlorococcus_B_sp003284185, s__Prochlorococcus_A_sp003282425, s__Eperythrozoon_A_wenyonii_A, s__GN02-872_sp003260325, s__SZUA-486_sp003251635, ...

Rooted; includes branch lengths.

## Checks

In [179]:
# checking overlap of genome metadata & phenotype data
overlap(brk, phy$tip.label, name)
cat('---\n')
overlap(brk, trt$tip.label, name)

intersect(x,y): 3195 
setdiff(x,y): 0 
setdiff(y,x): 20165 
union(x,y): 23360 
---
intersect(x,y): 3195 
setdiff(x,y): 0 
setdiff(y,x): 20165 
union(x,y): 23360 


# Preprocessing counts

In [180]:
# filtering low count taxa
brk %>% unique_n('No. of taxa', name)
brk_f = brk %>% 
    mutate.(mean_abund = mean(Abundance),
               by = c(name)) %>%
    filter.(mean_abund > 1e-3) %>%
    select.(Sample, name, Abundance) %>%
    pivot_wider(names_from=Sample, values_from=Abundance, values_fill=0) %>%
    as.data.frame
rownames(brk_f) = brk_f$name
brk_f$name = NULL
brk_f 

No. of unique No. of taxa: 3195 


,SID713A004.11.0.0,SID713A009.11.90.0,SID713A010.11.0.0,SID713A012.11.0.0,SID713A012.11.90.0,SID713A033.11.0.0,SID713A037.11.90.0,SID713A051.11.0.0,SID713B002.11.0.0,SID713B012.11.0.0,⋯,YSZC12003_37753,YSZC12003_37754,YSZC12003_37797,YSZC12003_37798,YSZC12003_37829,YSZC12003_37830,YSZC12003_37877,YSZC12003_37878,YSZC12003_37879,YSZC12003_37880
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s__Eubacterium_G_ventriosum,0.00028,0.00593,0.00231,0.00055,0e+00,0.00081,0.00000,0.00133,0.00000,0.00092,⋯,0e+00,0.00331,0.00142,0.00122,0.00027,0.00149,0.00531,0.00085,0.00037,0
s__Prevotella_sp000436035,0.00305,0.00000,0.00000,0.00265,5e-04,0.00224,0.00247,0.00135,0.01531,0.04421,⋯,4e-04,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
s__Megamonas_funiformis,0,0,0,0,0,0,0,0,0,0,⋯,0.00000,0,0,0.00000,0,0.00000,0,0.0000,0,0
s__Alistipes_sp900290115,0,0,0,0,0,0,0,0,0,0,⋯,0.00023,0,0,0.00196,0,0.13202,0,0.0012,0,0


In [181]:
# checking sample overlap
overlap(colnames(brk_f), meta %>% pull.(Sample))

intersect(x,y): 1929 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 1929 


# Prune tree & make matrix

## Phylo

In [182]:
# pruning
to_rm = setdiff(phy$tip.label, rownames(brk_f))
phy_f = ape::drop.tip(phy, to_rm)
phy_f


Phylogenetic tree with 198 tips and 197 internal nodes.

Tip labels:
	s__Methanobrevibacter_A_smithii, s__Ruminococcus_E_bromii_B, s__Ruminococcus_E_bromii, s__Ruminococcus_E_sp900314705, s__GCA-900066995_sp900291955, s__CAG-177_sp002451755, ...
Node labels:
	100.0, 1.0, 0.0, 0.0, 3.0, 18.0, ...

Rooted; includes branch lengths.

In [183]:
# to correlation matrix
phy_fm = cophenetic(phy_f) 
phy_fm = scales::rescale(phy_fm)
phy_fm = 1 - phy_fm[rownames(brk_f),rownames(brk_f)]
phy_fm

,s__Eubacterium_G_ventriosum,s__Prevotella_sp000436035,s__Prevotella_sp002440225,s__Acetatifactor_sp900066365,s__Bacteroides_A_coprocola,s__Bacteroides_sp002491635,s__Bifidobacterium_bifidum,s__CAG-180_sp000432435,s__Bacteroides_stercoris,s__Bacteroides_ovatus,⋯,s__Prevotella_stercorea,s__Bacteroides_B_sartorii,s__Blautia_A_sp900066205,s__Bacteroides_faecis,s__Bacteroides_B_dorei,s__Coprococcus_eutactus_A,s__Ruminiclostridium_E_siraeum,s__CAG-279_sp000437795,s__Megamonas_funiformis,s__Alistipes_sp900290115
s__Eubacterium_G_ventriosum,1.0000000,0.2403022,0.2445365,0.8133593,0.3124905,0.3061080,0.3120918,0.6088624,0.3077979,0.3116527,⋯,0.2500477,0.3180304,0.8163858,0.3134667,0.3178585,0.8112827,0.5912951,0.2997398,0.5503461,0.2832872
s__Prevotella_sp000436035,0.2403022,1.0000000,0.9800840,0.2362373,0.8383808,0.8319983,0.1310030,0.2348291,0.8336882,0.8375430,⋯,0.9123807,0.8439207,0.2392639,0.8393570,0.8437488,0.2265276,0.2172618,0.7912784,0.3002387,0.6293608
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
s__Megamonas_funiformis,0.5503461,0.3002387,0.3044730,0.5462813,0.3724270,0.3660445,0.3720284,0.5448730,0.3677344,0.3715892,⋯,0.3099842,0.3779669,0.5493078,0.3734032,0.3777950,0.5365715,0.5273057,0.3596763,1.0000000,0.3432237
s__Alistipes_sp900290115,0.2832872,0.6293608,0.6335951,0.2792224,0.7015491,0.6951666,0.1739880,0.2778141,0.6968565,0.7007113,⋯,0.6391064,0.7070890,0.2822489,0.7025253,0.7069171,0.2695126,0.2602468,0.6887985,0.3432237,1.0000000


## Trait

In [195]:
# pruning
to_rm = setdiff(trt$tip.label, rownames(brk_f))
trt_f = ape::drop.tip(trt, to_rm)
trt_f


Phylogenetic tree with 198 tips and 197 internal nodes.

Tip labels:
	s__Escherichia_coli, s__Escherichia_coli_C, s__Escherichia_coli_D, s__Escherichia_dysenteriae, s__Escherichia_flexneri, s__Klebsiella_pneumoniae, ...

Rooted; includes branch lengths.

In [196]:
# to correlation matrix
trt_fm = cophenetic(trt_f) 
trt_fm = scales::rescale(trt_fm)
trt_fm = 1 - trt_fm[rownames(brk_f),rownames(brk_f)]
trt_fm

,s__Eubacterium_G_ventriosum,s__Prevotella_sp000436035,s__Prevotella_sp002440225,s__Acetatifactor_sp900066365,s__Bacteroides_A_coprocola,s__Bacteroides_sp002491635,s__Bifidobacterium_bifidum,s__CAG-180_sp000432435,s__Bacteroides_stercoris,s__Bacteroides_ovatus,⋯,s__Prevotella_stercorea,s__Bacteroides_B_sartorii,s__Blautia_A_sp900066205,s__Bacteroides_faecis,s__Bacteroides_B_dorei,s__Coprococcus_eutactus_A,s__Ruminiclostridium_E_siraeum,s__CAG-279_sp000437795,s__Megamonas_funiformis,s__Alistipes_sp900290115
s__Eubacterium_G_ventriosum,1.0000000,0.2488917,0.2488917,0.5165172,0.2488917,0.2488917,0.3384942,0.3384942,0.2488917,0.2488917,⋯,0.2488917,0.2488917,0.3384942,0.2488917,0.2488917,0.3384942,0.3384942,0.2488917,0.1731299,0.2488917
s__Prevotella_sp000436035,0.2488917,1.0000000,0.6619839,0.2488917,0.5283590,0.5283590,0.2488917,0.2488917,0.5283590,0.5283590,⋯,0.5947696,0.5283590,0.2488917,0.5283590,0.5283590,0.2488917,0.2488917,0.4693376,0.1731299,0.4693376
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
s__Megamonas_funiformis,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,⋯,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,0.1731299,1.0000000,0.1731299
s__Alistipes_sp900290115,0.2488917,0.4693376,0.4693376,0.2488917,0.4693376,0.4693376,0.2488917,0.2488917,0.4693376,0.4693376,⋯,0.4693376,0.4693376,0.2488917,0.4693376,0.4693376,0.2488917,0.2488917,0.6349011,0.1731299,1.0000000


# Features

* `X_train.npy`
  * sample x taxon
* `Y_train.npy`
  * sample (target variable)
* `X_test.npy`
  * sample x taxon
* `Y_test.npy`
  * sample (target variable)
* `c.npy`
  * taxon x taxon

## Train-test split

In [186]:
# random sampling
x = brk_f %>% colnames
x_train = sample(x, length(x) * 0.8)
x_test = setdiff(x, x_train)
x_train %>% length
x_test %>% length

[1] 1543

[1] 386

In [187]:
# train
m_train = brk_f %>%
    t %>%
    .[x_train,] 
m_train

,s__Eubacterium_G_ventriosum,s__Prevotella_sp000436035,s__Prevotella_sp002440225,s__Acetatifactor_sp900066365,s__Bacteroides_A_coprocola,s__Bacteroides_sp002491635,s__Bifidobacterium_bifidum,s__CAG-180_sp000432435,s__Bacteroides_stercoris,s__Bacteroides_ovatus,⋯,s__Prevotella_stercorea,s__Bacteroides_B_sartorii,s__Blautia_A_sp900066205,s__Bacteroides_faecis,s__Bacteroides_B_dorei,s__Coprococcus_eutactus_A,s__Ruminiclostridium_E_siraeum,s__CAG-279_sp000437795,s__Megamonas_funiformis,s__Alistipes_sp900290115
YSZC12003_36675,0.00027,0,0,0.00097,0,0.00017,0,0.08107,0.00020,0.00113,⋯,0,0.00040,0.00255,0.00043,0.00114,0.00108,0.00018,0,0,0.00968
YSZC12003_35391,0.00089,0,0,0.00169,0,0.00163,0,0.00000,0.00065,0.00869,⋯,0,0.00186,0.00659,0.00089,0.07277,0.01286,0.00021,0,0,0.00000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
SID530154,4e-04,0.0000,0.00000,0.00028,0,0.00000,0.00037,0.00000,0.00000,0.00000,⋯,0.0000,0.00000,0.01863,0.00000,0.0000,0.00000,0.00000,0.00000,0,0
YSZC12003_37315,0e+00,0.0011,0.00157,0.00116,0,0.00184,0.00000,0.01681,0.00038,0.00141,⋯,0.0046,0.00084,0.00046,0.00096,0.0412,0.00047,0.00025,0.00038,0,0


In [188]:
# test
m_test = brk_f %>%
    t %>%
    .[x_test,] 
m_test

,s__Eubacterium_G_ventriosum,s__Prevotella_sp000436035,s__Prevotella_sp002440225,s__Acetatifactor_sp900066365,s__Bacteroides_A_coprocola,s__Bacteroides_sp002491635,s__Bifidobacterium_bifidum,s__CAG-180_sp000432435,s__Bacteroides_stercoris,s__Bacteroides_ovatus,⋯,s__Prevotella_stercorea,s__Bacteroides_B_sartorii,s__Blautia_A_sp900066205,s__Bacteroides_faecis,s__Bacteroides_B_dorei,s__Coprococcus_eutactus_A,s__Ruminiclostridium_E_siraeum,s__CAG-279_sp000437795,s__Megamonas_funiformis,s__Alistipes_sp900290115
SID713A009.11.90.0,0.00593,0.00000,0.00000,0.00642,0.00051,0.00099,0,0.00655,0.00109,0.00438,⋯,0.00000,0.00288,0.00099,0.00075,0.00900,0.00088,0.00000,0.00000,0,0
SID713A051.11.0.0,0.00133,0.00135,0.00267,0.00246,0.00140,0.00064,0,0.00000,0.00030,0.00443,⋯,0.00056,0.00239,0.00151,0.02865,0.06959,0.00794,0.01175,0.00556,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YSZC12003_37652,0.00135,0e+00,0.00000,0.00074,0.00000,0.00000,0.00000,0,0.00000,0.00565,⋯,0.00000,0.00135,0.00081,0.00254,0.05108,0.00531,0.00116,0,0,0.00018
YSZC12003_37753,0.00000,4e-04,0.00079,0.00298,0.00041,0.00024,0.00894,0,0.00039,0.00327,⋯,0.00312,0.00000,0.00097,0.00027,0.00215,0.00000,0.00041,0,0,0.00023


In [166]:
# # target: westernized
# x = meta %>%
#     select.(Sample, westernized) %>%
#     mutate.(westernized = westernized %>% as.numeric) %>%
#     as.data.frame
# rownames(x) = x$Sample
# x$Sample = NULL

# target_train = x[rownames(m_train),]
# target_train %>% table
# target_test = x[rownames(m_test),]
# target_test %>% table

.
   0    1 
 415 2263 

.
  0   1 
103 567 

In [206]:
# target: age
x = meta %>%
    select.(Sample, age) %>%
    mutate.(age= age %>% as.numeric %>% scale) %>%
    as.data.frame
rownames(x) = x$Sample
x$Sample = NULL

target_train = x[rownames(m_train),]
target_train %>% summary
target_test = x[rownames(m_test),]
target_test %>% summary

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-1.307056 -0.962650 -0.194362  0.005217  0.971317  3.355659 

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-1.30706 -0.98914 -0.08839 -0.02086  0.91833  1.81908 

## Writing

In [207]:
work_dir = '/ebio/abt3_projects/software/dev/tmp/MDeep/data/globalMG/'
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/software/dev/tmp/MDeep/data/globalMG/ 


In [208]:
F = file.path(work_dir, 'X_train.tsv')
write_table(m_train, F)

File written: /ebio/abt3_projects/software/dev/tmp/MDeep/data/globalMG//X_train.tsv 


In [209]:
F = file.path(work_dir, 'Y_train.tsv')
write_table(target_train, F)

File written: /ebio/abt3_projects/software/dev/tmp/MDeep/data/globalMG//Y_train.tsv 


In [210]:
F = file.path(work_dir, 'c.tsv')
write_table(phy_fm, F)

File written: /ebio/abt3_projects/software/dev/tmp/MDeep/data/globalMG//c.tsv 


In [198]:
D = file.path(dirname(work_dir), 'globalMG_trt')
F = file.path(D, 'c.tsv')
write_table(trt_fm, F)

File written: /ebio/abt3_projects/software/dev/tmp/MDeep/data/globalMG_trt/c.tsv 


# MDeep

## Convert to numpy

## Train: age

```
(mdeep) @ rick:/ebio/abt3_projects/software/dev/tmp/MDeep
python3 src/MDeep.py --train --data_dir data/globalMG --model_dir model --outcome_type continous  --batch_size 40 --max_epoch 2000 --learning_rate 5e-3 --dropout_rate 0.5 --window_size 8 8 8 --kernel_size 64 64 32 --strides 4 4 4
```

```
(mdeep) @ rick:/ebio/abt3_projects/software/dev/tmp/MDeep
python3 src/MDeep.py --train --data_dir data/globalMG_trt --model_dir model1 --outcome_type continous  --batch_size 50 --max_epoch 2000 --learning_rate 5e-3 --dropout_rate 0.5 --window_size 8 8 8 --kernel_size 64 64 32 --strides 4 4 4
```

```
(mdeep) @ rick:/ebio/abt3_projects/software/dev/tmp/MDeep
python3 src/MDeep.py --train --data_dir data/globalMG --model_dir model --outcome_type continous  --batch_size 40 --max_epoch 2000 --learning_rate 5e-3 --dropout_rate 0.5 --window_size 8 8 8 --kernel_size 64 64 32 --strides 4 4 4
```

## Train: westernized

```
(mdeep) @ rick:/ebio/abt3_projects/software/dev/tmp/MDeep
python3 src/MDeep.py --train --data_dir data/globalMG --model_dir model --outcome_type binary --batch_size 32 --max_epoch 500 --learning_rate 1e-4 --dropout_rate 0.5 --window_size 128 4 --kernel_size 32 32 --strides 64 2
```

```
(mdeep) @ rick:/ebio/abt3_projects/software/dev/tmp/MDeep
python3 src/MDeep.py --train --data_dir data/globalMG_trt --model_dir model --outcome_type binary --batch_size 32 --max_epoch 500 --learning_rate 1e-4 --dropout_rate 0.5 --window_size 128 4 --kernel_size 32 32 --strides 64 2
```

## Test

```
python3 src/MDeep.py --test --test_file data/globalMG/X_test.npy --correlation_file data/globalMG/c.npy --result_dir result/globalMG --model_dir model --outcome_type binary --batch_size 32 --max_epoch 500 --learning_rate 1e-4 --dropout_rate 0.5 --window_size 128 4 --kernel_size 32 32 --strides 64 2
```

# Sandbox

```
python3 src/MDeep.py --train --data_dir data/Malawiantwin_pairs --model_dir model --outcome_type continous --batch_size 16 --max_epoch 200 --learning_rate 5e-3 --dropout_rate 0.5 --window_size 8 8 8 --kernel_size 64 64 32 --strides 4 4 4
```